## Data PreProcess
![](./day1.jpg)

## 1. 导入numpy 和 pandas

In [1]:
import pandas as pd
import numpy as np

/Users/shengli/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## 2. 读取Data.csv，展示若干数据

In [2]:
df = pd.read_csv("../datasets/Data.csv")

In [3]:
df

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


### 2.1 截取dataframe所有行，列从开始到倒数第二列

In [4]:
X = df.iloc[:,:-1].values

### 2.2 截取dataframe第三列的所有row

In [5]:
Y = df.iloc[:,3].values

## 3. 处理丢失数据, Imputer, axis=0表示按列进行

In [6]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = "NaN", strategy = "mean", axis = 0)
imputer = imputer.fit(X[:, 1:3]) ## country and salary 
X[:,1:3] = imputer.transform(X[:,1:3]) ## 对第二列 到 第四列进行missing data转换, 仅数值列可以

#### Imputer用一些统计数据（例如平均值，中位数......）填充缺失值。为了避免交叉验证期间的数据泄漏，它在fit调用时存储训练数据的统计信息，将这些数据用到测试集合（transform）。

#### https://stackoverflow.com/questions/46691596/why-does-sklearn-imputer-need-to-fit 对于为什么需要fit的解释

In [7]:
X

array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, 63777.77777777778],
       ['France', 35.0, 58000.0],
       ['Spain', 38.77777777777778, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)

## 4. 分类数据的编码 -> Country, Purchased （YES／NO）

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

### 4.1 对第一列Country 进行 LabelEncoder

In [9]:
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0]) 
X

array([[0, 44.0, 72000.0],
       [2, 27.0, 48000.0],
       [1, 30.0, 54000.0],
       [2, 38.0, 61000.0],
       [1, 40.0, 63777.77777777778],
       [0, 35.0, 58000.0],
       [2, 38.77777777777778, 52000.0],
       [0, 48.0, 79000.0],
       [1, 50.0, 83000.0],
       [0, 37.0, 67000.0]], dtype=object)

### 4.2 创建一个虚拟变量， OneHotEncoder
#### https://www.cnblogs.com/king-lps/p/7846414.html

In [10]:
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()
labelencoder_Y = LabelEncoder()
Y =  labelencoder_Y.fit_transform(Y)

In [11]:
X[0]

array([1.0e+00, 0.0e+00, 0.0e+00, 4.4e+01, 7.2e+04])

In [12]:
Y

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

## 5. 把数据分为训练集和测试集的

In [13]:
from sklearn.cross_validation import train_test_split

/Users/shengli/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( X , Y , test_size = 0.2, random_state = 0)

## 6.归一化 z-score

In [15]:
from sklearn.preprocessing import StandardScaler

In [18]:
sc_X = StandardScaler()

In [19]:
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

In [20]:
X_train

array([[-1.        ,  2.64575131, -0.77459667,  0.26306757,  0.12381479],
       [ 1.        , -0.37796447, -0.77459667, -0.25350148,  0.46175632],
       [-1.        , -0.37796447,  1.29099445, -1.97539832, -1.53093341],
       [-1.        , -0.37796447,  1.29099445,  0.05261351, -1.11141978],
       [ 1.        , -0.37796447, -0.77459667,  1.64058505,  1.7202972 ],
       [-1.        , -0.37796447,  1.29099445, -0.0813118 , -0.16751412],
       [ 1.        , -0.37796447, -0.77459667,  0.95182631,  0.98614835],
       [ 1.        , -0.37796447, -0.77459667, -0.59788085, -0.48214934]])

In [21]:
X_test

array([[ 0.,  0.,  0., -1., -1.],
       [ 0.,  0.,  0.,  1.,  1.]])

### Done